# Few DynamoDB Samples using boto3

## A Sample showing deletion of Items

In [63]:
from boto3.dynamodb.conditions import Key, Attr
import boto3

dynamodb = boto3.resource('dynamodb', region_name='us-east-1')

table = dynamodb.Table('Time-Log')

fe = Attr('FileName').eq('REPLACE FILE NAME.pdf')

response = table.scan(
        FilterExpression=fe)

with table.batch_writer() as batch:   
    for item in response['Items']:
        print(item['Id'])
        batch.delete_item(Key={'Id':item['Id']})

# Sample function showing Cleanup of tables

In [ ]:
def _cleanExistingDynamoValues(table_name, file_name):
        from boto3.dynamodb.conditions import Key, Attr

        dynamodb = boto3.resource('dynamodb', region_name='us-east-1', config = config)

        table = dynamodb.Table(table_name)

        print('Deleting items pertaining to File Name -- ', file_name)
        
        fe = Attr('FileName').eq(file_name)
        pe = "Id"
        try :
            response = table.scan(
                    FilterExpression=fe,
                    ProjectionExpression=pe
                    )

            with table.batch_writer() as batch:   
                for item in response['Items']:
                    print('Deleting ', item['Id'])
                    batch.delete_item(Key={'Id':item['Id']})
                    
        except ClientError as e:
            if e.response['Error']['Code'] in ["ThrottlingException", "ProvisionedThroughputExceededException"]:
                msg = f"DynamoDB ] Write Failed from DynamoDB, Throttling Exception [{e}] [{traceback.format_exc()}]"
                logging.warning(msg)
                raise e

            else:
                msg = f"DynamoDB Write Failed from DynamoDB Exception [{e}] [{traceback.format_exc()}]"
                logging.error(msg)
                raise e

        except Exception as e:
            msg = f"DynamoDB Write Failed from DynamoDB Exception [{e}] [{traceback.format_exc()}]"
            logging.error(msg)
            raise Exception(e)

In [31]:
from botocore.client import Config

In [32]:
config = Config(
            retries = dict(
                max_attempts = 5
            )
        )

In [34]:
_cleanExistingDynamoValues('Time-Log', 'CCU GOLDEN XXXXX.pdf')

Deleting items pertaining to File Name --  CCU GOLDEN CREEK 1-1-26TFH CMP (01-26-2020).pdf
Deleting  d944447b-5c39-4deb-9944-ac35c16c8632
Deleting  8c9fd165-d9ef-487f-8f56-2bc01be6fa03
Deleting  2d34ef8e-7631-4c08-a234-03b0b87b1cef
Deleting  46829732-3e2b-41af-b306-7e4143ff1b0a
Deleting  a5ce9117-7ec7-4ce6-a133-a146fa8dce3c
Deleting  a902204d-b3a2-47f5-b49e-a3666422988e
Deleting  97e53010-01e5-45d2-aed8-039f28981122
Deleting  c8147448-f1da-413b-acd5-fd84cb7d6094
Deleting  e90ae704-342c-4e67-aa63-5a803f076fb9
Deleting  ff8ba127-dd5c-4bf1-9ce1-2285b27fcc6d


## Sample showing pulling in all tables - Pay attention, there is no limit of 100 here

db = boto3.resource('dynamodb', region_name="us-east-1")
tables = list([x.name for x in db.tables.all()])
print(tables)

# Sample Function showing creating tables in DynamoDB

In [55]:
def _writeToDynamoDB(dd_table_name):

        # Get the service resource.
        dynamodb = boto3.resource('dynamodb', config=config)
        dynamodb_client = boto3.client('dynamodb', config=config)
        
        dd_table_name = dd_table_name \
                        .replace(" ", "-") \
                        .replace("(", "-") \
                        .replace(")", "-") \
                        .replace("&", "-") \
                        .replace(",", " ") \
                        .replace(":", "-") \
                        .replace('/', '--') \
                        .replace("#", 'No') \
                        .replace('"', 'Inch') 

        if len(dd_table_name) <= 3:
            dd_table_name = dd_table_name + '-xxxx'
            
        print("DynamoDB table name is {}".format(dd_table_name))

        # Create the DynamoDB table.
        try:
            
            existing_tables = list([x.name for x in dynamodb.tables.all()])

            # existing_tables = dynamodb_client.list_tables()['TableNames']

            if dd_table_name not in existing_tables:
                table = dynamodb.create_table(
                    TableName=dd_table_name,
                    KeySchema=[
                        {
                            'AttributeName': 'Id',
                            'KeyType': 'HASH'
                        }
                    ],
                    AttributeDefinitions=[
                        {
                            'AttributeName': 'Id',
                            'AttributeType': 'S'
                        },
                    ],
                    BillingMode='PAY_PER_REQUEST',
                )
                # Wait until the table exists, this will take a minute or so
                table.meta.client.get_waiter('table_exists').wait(TableName=dd_table_name)
                # Print out some data about the table.
                print("Table successfully created. Item count is: " + str(table.item_count))
        # except dynamodb_client.exceptions.ResourceInUseException:
        except ClientError as e:
            if e.response['Error']['Code'] in ["ThrottlingException", "ProvisionedThroughputExceededException"]:
                msg = f"DynamoDB ] Write Failed from DynamoDB, Throttling Exception [{e}] [{traceback.format_exc()}]"
                logging.warning(msg)
                raise e
            else:
                msg = f"DynamoDB Write Failed from DynamoDB Exception [{e}] [{traceback.format_exc()}]"
                logging.error(msg)
                raise e

        except Exception as e:
             msg = f"DynamoDB Write Failed from DynamoDB Exception [{e}] [{traceback.format_exc()}]"
             logging.error(msg)
             raise Exception(e)
        
        

### Create some dummy tables

In [ ]:
for i in range(100):
    _writeToDynamoDB("table " + str(i))

In [57]:
db = boto3.resource('dynamodb', region_name="us-east-1")
tables = list([x.name for x in db.tables.all()])
print(tables)

['Casing-Strings', 'Drill-Bits', 'Drill-String-Components', 'Drilling-Parameters', 'Head-Count----Manhours', 'Nxxxx', 'Survey-Data', 'Time-Log', 'key-values', 'table-0', 'table-1', 'table-10', 'table-11', 'table-12', 'table-13', 'table-14', 'table-15', 'table-16', 'table-17', 'table-18', 'table-19', 'table-2', 'table-20', 'table-21', 'table-22', 'table-23', 'table-24', 'table-25', 'table-26', 'table-27', 'table-28', 'table-29', 'table-3', 'table-30', 'table-31', 'table-32', 'table-33', 'table-34', 'table-35', 'table-36', 'table-37', 'table-38', 'table-39', 'table-4', 'table-40', 'table-41', 'table-42', 'table-43', 'table-44', 'table-45', 'table-46', 'table-47', 'table-48', 'table-49', 'table-5', 'table-50', 'table-51', 'table-52', 'table-53', 'table-54', 'table-55', 'table-56', 'table-57', 'table-58', 'table-59', 'table-6', 'table-60', 'table-61', 'table-62', 'table-63', 'table-64', 'table-65', 'table-66', 'table-67', 'table-68', 'table-69', 'table-7', 'table-70', 'table-71', 'table-72

In [65]:
### Check if you are getting more than 100 tables

In [59]:
print(len(tables))

110
